In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score

# ----- Step 1: Load and Prepare Weekly Data -----
df_weekly = pd.read_csv("/content/weekly_train.csv")
df_weekly['Week'] = pd.to_datetime(df_weekly['week'])  # Convert 'week' column to datetime
df_weekly = df_weekly.sort_values(by=['fdr_Id', 'Week'])

# Shift the Fault_Occurred column to create target labels
df_weekly['target'] = df_weekly.groupby('fdr_Id')['Fault_Occurred'].shift(-1)
df_weekly = df_weekly.dropna(subset=['target'])
df_weekly['target'] = df_weekly['target'].astype(int)

# ----- Step 2: Define Features and Normalize -----
exclude_cols = ['fdr_Id', 'Week', 'Fault_Occurred', 'target', 'week']
features = [col for col in df_weekly.columns if col not in exclude_cols]
scaler = MinMaxScaler()
df_weekly[features] = scaler.fit_transform(df_weekly[features])

# ----- Step 3: Create Feeder-Wise Sequences -----
def create_sequences_by_feeder(df, seq_length=3):
    X_list, y_list = [], []
    feeders = df['fdr_Id'].unique()
    for feeder in feeders:
        feeder_df = df[df['fdr_Id'] == feeder].reset_index(drop=True)
        for i in range(len(feeder_df) - seq_length):
            seq_X = feeder_df[features].iloc[i:i+seq_length].values
            seq_y = feeder_df['target'].iloc[i + seq_length]
            X_list.append(seq_X)
            y_list.append(seq_y)
    return torch.tensor(np.array(X_list), dtype=torch.float32), torch.tensor(np.array(y_list), dtype=torch.float32)

seq_length = 10
X, y = create_sequences_by_feeder(df_weekly, seq_length=seq_length)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X, y = X.to(device), y.to(device)

# ----- Step 4: Split Data and Create DataLoaders -----
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

batch_size = 64
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

# ----- Step 5: Define the LSTM Model -----
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        x = hidden[-1]
        x = self.fc(x)
        return torch.sigmoid(x)

# Instantiate model
model = LSTMModel(input_dim=X.shape[2]).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----- Step 6: Train the Model -----
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_X).squeeze()
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

# ----- Step 7: Evaluate the Model -----
model.eval()
y_pred_list, y_test_list = [], []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        output = model(batch_X).squeeze()
        y_pred_list.extend(output.cpu().numpy())
        y_test_list.extend(batch_y.cpu().numpy())

y_pred_binary = (np.array(y_pred_list) > 0.5).astype(int)
y_test_cpu = np.array(y_test_list)

precision = precision_score(y_test_cpu, y_pred_binary)
recall = recall_score(y_test_cpu, y_pred_binary)
f1 = f1_score(y_test_cpu, y_pred_binary)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# ----- Step 8: Predict Feeders Going Down in Target Week -----
target_week = pd.to_datetime("2025-01-06")
df_filtered = df_weekly[df_weekly['Week'] < target_week]

feeder_predictions = []
feeders = df_filtered['fdr_Id'].unique()

with torch.no_grad():
    for feeder in feeders:
        feeder_df = df_filtered[df_filtered['fdr_Id'] == feeder].reset_index(drop=True)
        if len(feeder_df) >= seq_length:
            seq_data = feeder_df[features].iloc[-seq_length:].values
            seq_tensor = torch.tensor(seq_data, dtype=torch.float32).unsqueeze(0).to(device)
            pred_prob = model(seq_tensor).item()
            feeder_predictions.append((feeder, pred_prob))

threshold = 0.24
down_feeders = {(f, prob) for f, prob in feeder_predictions if prob > threshold}

# ----- Step 9: Compare Predictions with Actual Data -----
actual_data = pd.read_csv("/content/weekly_test.csv")
actual_data['Week'] = pd.to_datetime(actual_data['week'])
actual_failures = set(actual_data[actual_data['Fault_Occurred'] == 1]['fdr_Id'])

predicted_failures = {f for f, _ in down_feeders}
true_positives = actual_failures & predicted_failures
false_negatives = actual_failures - predicted_failures
false_positives = predicted_failures - actual_failures

precision = len(true_positives) / (len(true_positives) + len(false_positives) + 1e-9)
recall = len(true_positives) / (len(true_positives) + len(false_negatives) + 1e-9)
f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)

print(f"Correct Predictions: {len(true_positives)}")
print(f"Missed Failures: {len(false_negatives)}")
print(f"Incorrect Predictions: {len(false_positives)}")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")


Epoch 1/10, Loss: 0.6260
Epoch 2/10, Loss: 0.5691
Epoch 3/10, Loss: 0.5620
Epoch 4/10, Loss: 0.5627
Epoch 5/10, Loss: 0.5562
Epoch 6/10, Loss: 0.5538
Epoch 7/10, Loss: 0.5519
Epoch 8/10, Loss: 0.5539
Epoch 9/10, Loss: 0.5497
Epoch 10/10, Loss: 0.5504
Precision: 0.5993, Recall: 0.5254, F1 Score: 0.5599
Correct Predictions: 113
Missed Failures: 16
Incorrect Predictions: 12
Precision: 0.9040, Recall: 0.8760, F1 Score: 0.8898


Newer Version

In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score

# ----- Step 1: Load and Prepare Weekly Data -----
df_weekly = pd.read_csv("/content/weekly_train.csv")
df_weekly['Week'] = pd.to_datetime(df_weekly['week'])  # Convert 'week' column to datetime
df_weekly = df_weekly.sort_values(by=['fdr_Id', 'Week'])

# Shift the Fault_Occurred column to create target labels
df_weekly['target'] = df_weekly.groupby('fdr_Id')['Fault_Occurred'].shift(-1)
df_weekly = df_weekly.dropna(subset=['target'])
df_weekly['target'] = df_weekly['target'].astype(int)

# ----- Step 2: Define Features and Normalize -----
exclude_cols = ['fdr_Id', 'Week', 'Fault_Occurred', 'target', 'week']
features = [col for col in df_weekly.columns if col not in exclude_cols]
scaler = MinMaxScaler()
df_weekly[features] = scaler.fit_transform(df_weekly[features])

# ----- Step 3: Create Feeder-Wise Sequences -----
def create_sequences_by_feeder(df, seq_length=3):
    X_list, y_list = [], []
    feeders = df['fdr_Id'].unique()
    for feeder in feeders:
        feeder_df = df[df['fdr_Id'] == feeder].reset_index(drop=True)
        for i in range(len(feeder_df) - seq_length):
            seq_X = feeder_df[features].iloc[i:i+seq_length].values
            seq_y = feeder_df['target'].iloc[i + seq_length]
            X_list.append(seq_X)
            y_list.append(seq_y)
    return torch.tensor(np.array(X_list), dtype=torch.float32), torch.tensor(np.array(y_list), dtype=torch.float32)

seq_length = 20
X, y = create_sequences_by_feeder(df_weekly, seq_length=seq_length)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X, y = X.to(device), y.to(device)

# ----- Step 4: Split Data and Create DataLoaders -----
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

batch_size = 64
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

# ----- Step 5: Define the LSTM Model -----
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        x = hidden[-1]
        x = self.fc(x)
        return torch.sigmoid(x)

# Instantiate model
model = LSTMModel(input_dim=X.shape[2]).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----- Step 6: Train the Model -----
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_X).squeeze()
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

# ----- Step 7: Evaluate the Model -----
model.eval()
y_pred_list, y_test_list = [], []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        output = model(batch_X).squeeze()
        y_pred_list.extend(output.cpu().numpy())
        y_test_list.extend(batch_y.cpu().numpy())

y_pred_binary = (np.array(y_pred_list) > 0.5).astype(int)
y_test_cpu = np.array(y_test_list)

precision = precision_score(y_test_cpu, y_pred_binary)
recall = recall_score(y_test_cpu, y_pred_binary)
f1 = f1_score(y_test_cpu, y_pred_binary)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# ----- Step 8: Predict Feeders Going Down for Each Week of January 2025 -----
weeks_to_predict = [
    pd.to_datetime("2025-01-06"),  # Week 1
    pd.to_datetime("2025-01-13"),  # Week 2
    pd.to_datetime("2025-01-20"),  # Week 3
    pd.to_datetime("2025-01-27"),  # Week 4
]

all_predictions = []

with torch.no_grad():
    for target_week in weeks_to_predict:
        df_filtered = df_weekly[df_weekly['Week'] < target_week]
        feeder_predictions = []
        feeders = df_filtered['fdr_Id'].unique()

        for feeder in feeders:
            feeder_df = df_filtered[df_filtered['fdr_Id'] == feeder].reset_index(drop=True)
            if len(feeder_df) >= seq_length:
                seq_data = feeder_df[features].iloc[-seq_length:].values
                seq_tensor = torch.tensor(seq_data, dtype=torch.float32).unsqueeze(0).to(device)
                pred_prob = model(seq_tensor).item()
                feeder_predictions.append({
                    'week': target_week.strftime('%Y-%m-%d'),
                    'fdr_Id': feeder,
                    'predicted_probability': pred_prob
                })

        all_predictions.extend(feeder_predictions)

# Convert predictions to DataFrame and save
predictions_df = pd.DataFrame(all_predictions)
predictions_df.to_csv("/content/predicted_feeder_failures_jan2025.csv", index=False)
print("Predictions saved to /content/predicted_feeder_failures_jan2025.csv")



Epoch 1/10, Loss: 0.6259
Epoch 2/10, Loss: 0.5625
Epoch 3/10, Loss: 0.5539
Epoch 4/10, Loss: 0.5500
Epoch 5/10, Loss: 0.5476
Epoch 6/10, Loss: 0.5479
Epoch 7/10, Loss: 0.5444
Epoch 8/10, Loss: 0.5449
Epoch 9/10, Loss: 0.5435
Epoch 10/10, Loss: 0.5410
Precision: 0.6027, Recall: 0.5411, F1 Score: 0.5702
Predictions saved to /content/predicted_feeder_failures_jan2025.csv


In [27]:
# ----- Step 9: Compare Predictions with Actual Data for Each Week -----
actual_data = pd.read_csv("/content/weekly_test.csv")
actual_data['Week'] = pd.to_datetime(actual_data['week'])

evaluation_results = []
threshold = 0.37  # Use the same threshold

for target_week in weeks_to_predict:
    week_str = target_week.strftime('%Y-%m-%d')

    # Filter predictions and actuals for this week
    week_predictions = predictions_df[predictions_df['week'] == week_str]
    predicted_failures = set(week_predictions[week_predictions['predicted_probability'] > threshold]['fdr_Id'])

    week_actuals = actual_data[actual_data['Week'] == target_week]
    actual_failures = set(week_actuals[week_actuals['Fault_Occurred'] == 1]['fdr_Id'])

    true_positives = actual_failures & predicted_failures
    false_negatives = actual_failures - predicted_failures
    false_positives = predicted_failures - actual_failures

    precision = len(true_positives) / (len(true_positives) + len(false_positives) + 1e-9)
    recall = len(true_positives) / (len(true_positives) + len(false_negatives) + 1e-9)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-9)

    evaluation_results.append({
        'week': week_str,
        'correct_predictions': len(true_positives),
        'missed_failures': len(false_negatives),
        'incorrect_predictions': len(false_positives),
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    })

    print(f"\nResults for {week_str}:")
    print(f"Correct Predictions: {len(true_positives)}")
    print(f"Missed Failures: {len(false_negatives)}")
    print(f"Incorrect Predictions: {len(false_positives)}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Save evaluation results to CSV
evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df.to_csv("/content/evaluation_results_jan2025.csv", index=False)
print("\nEvaluation results saved to /content/evaluation_results_jan2025.csv")

# ---- NEW PART: Calculate and print average F1 Score ----
average_f1 = evaluation_df['f1_score'].mean()
print(f"\nAverage F1 Score over all weeks: {average_f1:.4f}")



Results for 2025-01-06:
Correct Predictions: 34
Missed Failures: 22
Incorrect Predictions: 28
Precision: 0.5484, Recall: 0.6071, F1 Score: 0.5763

Results for 2025-01-13:
Correct Predictions: 37
Missed Failures: 26
Incorrect Predictions: 25
Precision: 0.5968, Recall: 0.5873, F1 Score: 0.5920

Results for 2025-01-20:
Correct Predictions: 44
Missed Failures: 23
Incorrect Predictions: 18
Precision: 0.7097, Recall: 0.6567, F1 Score: 0.6822

Results for 2025-01-27:
Correct Predictions: 40
Missed Failures: 20
Incorrect Predictions: 22
Precision: 0.6452, Recall: 0.6667, F1 Score: 0.6557

Evaluation results saved to /content/evaluation_results_jan2025.csv

Average F1 Score over all weeks: 0.6265
